# Clase 3 – Ingesta y Capa Bronce

En esta notebook se inicia la construcción del pipeline de datos meteorológicos, trabajando con los archivos crudos provistos por el SMN.


## 1. Librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path


## 2. Configuración de paths y carpetas

In [2]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'

# Crear carpetas si no existen
for path in [BRONCE_DIR]:
    path.mkdir(parents=True, exist_ok=True)


## 3. Lectura del archivo de estaciones

In [3]:
# Ruta del archivo
archivo_estaciones = RAW_DIR / 'estaciones' / 'estaciones_smn.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_estaciones, "r", encoding="latin1") as f:
    lines = f.readlines()[2:]

# Expresión regular para extraer campos:
pattern = re.compile(
    r"^(?P<nombre>.+?)\s{2,}(?P<provincia>.+?)\s{2,}(?P<lat_gr>-?\d+)\s+(?P<lat_min>\d+)\s+(?P<lon_gr>-?\d+)\s+(?P<lon_min>\d+)\s+(?P<altura_m>\d+)\s+(?P<numero>\d+)\s+(?P<numero_oaci>\S+)\s*$"
)

# Extraer los datos
data = []
for line in lines:
    match = pattern.match(line)
    if match:
        data.append(match.groupdict())

# Crear DataFrame
df_estaciones = pd.DataFrame(data)

# Conversión de tipos
df_estaciones[['lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero']] = df_estaciones[[
    'lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero'
]].apply(pd.to_numeric)

# Filtrar estaciones de Misiones
df_misiones = df_estaciones[df_estaciones['provincia'].str.upper().str.strip() == 'MISIONES']
df_misiones[['nombre', 'provincia', 'numero', 'numero_oaci']]

,nombre,provincia,numero,numero_oaci
77,BERNARDO DE IRIGOYEN AERO,MISIONES,87163,SATI
78,IGUAZU AERO,MISIONES,87097,SARI
79,OBERA,MISIONES,87187,SATO
80,POSADAS AERO,MISIONES,87178,SARP


## 4. Selección de estaciones de Misiones

In [4]:
df_misiones = df_estaciones[df_estaciones['provincia'].str.upper() == 'MISIONES']
df_misiones[['nombre', 'provincia', 'numero', 'numero_oaci']]


,nombre,provincia,numero,numero_oaci
77,BERNARDO DE IRIGOYEN AERO,MISIONES,87163,SATI
78,IGUAZU AERO,MISIONES,87097,SARI
79,OBERA,MISIONES,87187,SATO
80,POSADAS AERO,MISIONES,87178,SARP


## 5. Lectura de un archivo horario de ejemplo

In [5]:
archivo_dato = RAW_DIR / 'datohorario' / 'datohorario20240601.txt'
df_dato = pd.read_csv(archivo_dato, sep=';', encoding='latin1')
df_dato.head()


,FECHA HORA TEMP HUM PNM DD FF NOMBRE
0,[HOA] [ºC] [%] [hPa] [gr] [km/hr...
1,01062024 0 14.2 82 1015.7 50 17 ...
2,01062024 1 14.3 80 1015.4 360 9 ...
3,01062024 2 14.1 86 1015.3 360 9 ...
4,01062024 3 14.1 87 1014.8 360 7 ...


## 6. Limpieza básica y detección de nulos

In [6]:
#df_dato.replace({9999.9: np.nan, -9999: np.nan}, inplace=True)
#df_dato.isna().sum()

# Contar valores a reemplazar antes de la limpieza
cant_9999_9 = (df_dato == 9999.9).sum().sum()
cant_neg9999 = (df_dato == -9999).sum().sum()

# Reemplazar por NaN
df_dato.replace({9999.9: np.nan, -9999: np.nan}, inplace=True)

# Imprimir resumen
print(f"Reemplazados {cant_9999_9} valores de 9999.9 y {cant_neg9999} valores de -9999 por NaN.")
print("Valores faltantes por columna luego del reemplazo:")
print(df_dato.isna().sum())


Reemplazados 0 valores de 9999.9 y 0 valores de -9999 por NaN.
Valores faltantes por columna luego del reemplazo:
FECHA     HORA  TEMP   HUM   PNM    DD    FF     NOMBRE                                                 0
dtype: int64


## 7. Filtro por estación de Misiones

In [7]:
archivo_dato = RAW_DIR / 'datohorario' / 'datohorario20240601.txt'

# Leer todas las líneas, omitiendo las dos primeras (encabezado y unidades)
with open(archivo_dato, "r", encoding="latin1") as f:
    lines = f.readlines()

# Detectar columnas separadas por múltiples espacios
columnas = re.split(r"\s{2,}", lines[0].strip())

# Leer datos
data = [
    re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
    for line in lines[1:]
    if len(line.strip()) > 0 and not line.isspace()
]

df_dato = pd.DataFrame(data, columns=columnas)

# Filtrar por estaciones de Misiones
df_dato["NOMBRE"] = df_dato["NOMBRE"].str.strip()
df_misiones_dia = df_dato[df_dato["NOMBRE"].isin(nombres_misiones)]
#print(df_misiones_dia.head())

# Mostrar todos los resultados (sin limitar con .head())
print(df_misiones_dia.to_string(index=False))


NameError: name 'nombres_misiones' is not defined

## 8. Exportación de archivos filtrados

In [ ]:
# Crear carpeta de salida si no existe
BRONCE_DIR.mkdir(parents=True, exist_ok=True)

# Definir la fecha (puede venir del nombre del archivo)
fecha = "20240601"  # o extraela dinámicamente si lo preferís

# Iterar por cada estación de Misiones
for nombre in nombres_misiones:
    nombre_clean = nombre.lower().replace(' ', '_')
    
    # Filtrar las filas de esa estación
    df_estacion = df_misiones_dia[df_misiones_dia["NOMBRE"] == nombre]
    
    # Definir archivos de salida con fecha al inicio
    salida_csv = BRONCE_DIR / f'{fecha}_{nombre_clean}.csv'
    salida_parquet = BRONCE_DIR / f'{fecha}_{nombre_clean}.parquet'
    
    # Exportar
    df_estacion.to_csv(salida_csv, index=False)
    df_estacion.to_parquet(salida_parquet, index=False)
    
    print(f"Exportado: {salida_csv.name} y {salida_parquet.name}")


## 9. Próximos pasos

- Extender este proceso a más días o meses.
- Organizar las salidas por carpeta `/bronce/{estacion}/{año}/`.
- Documentar el diccionario de variables en `metadata/`.


In [ ]:

# 🟡 Paso 1: Leer archivo de estaciones y filtrar las de Misiones
# --------------------------------------------------------------
import pandas as pd
import re
from pathlib import Path

# Definir carpeta de datos crudos
RAW_DIR = Path("data/raw")

# Leer archivo de estaciones como texto, ignorando las 2 primeras filas
archivo_estaciones = RAW_DIR / "estaciones" / "estaciones_smn.txt"
with open(archivo_estaciones, encoding="latin1") as f:
    lines = f.readlines()[2:]

# Aplicar regex para extraer campos de cada estación
pattern = re.compile(
    r"^(?P<nombre>.+?)\s{2,}(?P<provincia>.+?)\s{2,}(?P<lat_gr>-?\d+)\s+(?P<lat_min>\d+)\s+(?P<lon_gr>-?\d+)\s+(?P<lon_min>\d+)\s+(?P<altura_m>\d+)\s+(?P<numero>\d+)\s+(?P<numero_oaci>\S+)\s*$"
)
data = [m.groupdict() for line in lines if (m := pattern.match(line))]

# Crear DataFrame y convertir tipos
df_estaciones = pd.DataFrame(data)
df_estaciones[['lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero']] = df_estaciones[[
    'lat_gr', 'lat_min', 'lon_gr', 'lon_min', 'altura_m', 'numero'
]].apply(pd.to_numeric)

# Filtrar estaciones de la provincia de Misiones
df_misiones = df_estaciones[df_estaciones['provincia'].str.upper().str.strip() == "MISIONES"]
nombres_misiones = df_misiones["nombre"].str.strip().unique()

print("Estaciones de Misiones:")
print(nombres_misiones)

# 🟡 Paso 2: Procesar archivo datohorario por fecha
# -------------------------------------------------
# Por cada archivo, cargar datos y filtrar por las estaciones de Misiones
from glob import glob

archivos_datos = sorted(glob(str(RAW_DIR / "datohorario*.txt")))
print(f"Se encontraron {len(archivos_datos)} archivos datohorario para procesar")

for archivo in archivos_datos:
    print(f"Procesando: {archivo}")

    # Leer el archivo como texto plano
    with open(archivo, encoding="latin1") as f:
        raw_lines = f.readlines()

    # Obtener nombre de columnas desde la primera línea
    header = raw_lines[0].strip()
    columnas = re.split(r"\s{2,}", header)

    # Procesar líneas de datos
    data = [re.split(r"\s{2,}", line.strip(), maxsplit=len(columnas)-1)
            for line in raw_lines[1:] if len(line.strip()) > 0 and not line.isspace()]

    # Crear DataFrame
    df_dato = pd.DataFrame(data, columns=columnas)

    # Filtrar por nombre de estación
    df_dato_misiones = df_dato[df_dato["NOMBRE"].str.strip().isin(nombres_misiones)]

    # Obtener fecha del archivo
    nombre_archivo = Path(archivo).stem  # ej: datohorario20240601
    fecha = nombre_archivo.replace("datohorario", "")

    # Exportar resultados
    out_csv = RAW_DIR / f"misiones_{fecha}.csv"
    out_parquet = RAW_DIR / f"misiones_{fecha}.parquet"
    df_dato_misiones.to_csv(out_csv, index=False)
    df_dato_misiones.to_parquet(out_parquet, index=False)

    print(f"Guardado CSV: {out_csv}")
    print(f"Guardado Parquet: {out_parquet}")
